# 🚀 Google Colab Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ogautier1980/sandbox-ml/blob/main/cours/13_systemes_recommandation/13_demo_collaborative_filtering.ipynb)

**Si vous exécutez ce notebook sur Google Colab**, exécutez la cellule suivante pour installer les dépendances.

In [ ]:
# Installation des dépendances (Google Colab uniquement)import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    print('📦 Installation des packages...')        # Packages ML de base    !pip install -q numpy pandas matplotlib seaborn scikit-learn        # Détection du chapitre et installation des dépendances spécifiques    notebook_name = '13_demo_collaborative_filtering.ipynb'  # Sera remplacé automatiquement        # Ch 06-08 : Deep Learning    if any(x in notebook_name for x in ['06_', '07_', '08_']):        !pip install -q torch torchvision torchaudio        # Ch 08 : NLP    if '08_' in notebook_name:        !pip install -q transformers datasets tokenizers        if 'rag' in notebook_name:            !pip install -q sentence-transformers faiss-cpu rank-bm25        # Ch 09 : Reinforcement Learning    if '09_' in notebook_name:        !pip install -q gymnasium[classic-control]        # Ch 04 : Boosting    if '04_' in notebook_name and 'boosting' in notebook_name:        !pip install -q xgboost lightgbm catboost        # Ch 05 : Clustering avancé    if '05_' in notebook_name:        !pip install -q umap-learn        # Ch 11 : Séries temporelles    if '11_' in notebook_name:        !pip install -q statsmodels prophet        # Ch 12 : Vision avancée    if '12_' in notebook_name:        !pip install -q ultralytics timm segmentation-models-pytorch        # Ch 13 : Recommandation    if '13_' in notebook_name:        !pip install -q scikit-surprise implicit        # Ch 14 : MLOps    if '14_' in notebook_name:        !pip install -q mlflow fastapi pydantic        print('✅ Installation terminée !')else:    print('ℹ️  Environnement local détecté, les packages sont déjà installés.')

# Chapitre 14 - Démonstration : Collaborative Filtering

Ce notebook illustre les différentes approches de **Collaborative Filtering** pour les systèmes de recommandation :

1. **Exploration du dataset MovieLens**
2. **User-Based Collaborative Filtering**
3. **Item-Based Collaborative Filtering**
4. **Matrix Factorization avec SVD**
5. **Évaluation et comparaison**
6. **Visualisation des embeddings**

Dataset : **MovieLens 100K** (100,000 ratings de 943 utilisateurs sur 1,682 films)

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
np.random.seed(42)

## 1. Chargement et Exploration du Dataset MovieLens

Nous utilisons le dataset MovieLens 100K disponible via la bibliothèque `surprise` ou téléchargeable directement.

In [ ]:
# Télécharger et charger MovieLens 100K
from surprise import Dataset
from surprise.model_selection import train_test_split as surprise_split

# Charger le dataset
data = Dataset.load_builtin('ml-100k')
ratings_df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'item_id', 'rating', 'timestamp'])

print(f"Dataset shape: {ratings_df.shape}")
print(f"\nNombre d'utilisateurs: {ratings_df['user_id'].nunique()}")
print(f"Nombre de films: {ratings_df['item_id'].nunique()}")
print(f"Nombre de ratings: {len(ratings_df)}")
print(f"\nRatings min/max: {ratings_df['rating'].min()}/{ratings_df['rating'].max()}")

ratings_df.head(10)

In [ ]:
# Statistiques descriptives
print("\n=== Statistiques des Ratings ===")
print(ratings_df['rating'].describe())

# Distribution des ratings
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Distribution des ratings
ratings_df['rating'].value_counts().sort_index().plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Distribution des Ratings')
axes[0].set_xlabel('Rating')
axes[0].set_ylabel('Nombre')

# Nombre de ratings par utilisateur
user_counts = ratings_df.groupby('user_id').size()
user_counts.hist(bins=50, ax=axes[1], color='coral')
axes[1].set_title('Ratings par Utilisateur')
axes[1].set_xlabel('Nombre de ratings')
axes[1].set_ylabel('Nombre d\'utilisateurs')
axes[1].axvline(user_counts.mean(), color='red', linestyle='--', label=f'Moyenne: {user_counts.mean():.1f}')
axes[1].legend()

# Nombre de ratings par film
item_counts = ratings_df.groupby('item_id').size()
item_counts.hist(bins=50, ax=axes[2], color='mediumseagreen')
axes[2].set_title('Ratings par Film')
axes[2].set_xlabel('Nombre de ratings')
axes[2].set_ylabel('Nombre de films')
axes[2].axvline(item_counts.mean(), color='red', linestyle='--', label=f'Moyenne: {item_counts.mean():.1f}')
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"\nRatings par utilisateur - Min: {user_counts.min()}, Max: {user_counts.max()}, Moyenne: {user_counts.mean():.1f}")
print(f"Ratings par film - Min: {item_counts.min()}, Max: {item_counts.max()}, Moyenne: {item_counts.mean():.1f}")

In [ ]:
# Analyser la sparsité de la matrice
n_users = ratings_df['user_id'].nunique()
n_items = ratings_df['item_id'].nunique()
n_ratings = len(ratings_df)

sparsity = 1 - (n_ratings / (n_users * n_items))
density = n_ratings / (n_users * n_items)

print(f"\n=== Sparsité de la Matrice User-Item ===")
print(f"Taille de la matrice: {n_users} users × {n_items} items = {n_users * n_items:,} cellules")
print(f"Ratings observés: {n_ratings:,}")
print(f"Densité: {density:.2%}")
print(f"Sparsité: {sparsity:.2%}")
print(f"\nEn moyenne, chaque utilisateur a raté seulement {density * n_items:.1f} films sur {n_items}.")

## 2. Préparation des Données

Nous créons la matrice user-item et effectuons un split train/test.

In [ ]:
# Créer des IDs numériques consécutifs pour indexing
user_ids = ratings_df['user_id'].unique()
item_ids = ratings_df['item_id'].unique()

user_id_map = {uid: idx for idx, uid in enumerate(user_ids)}
item_id_map = {iid: idx for idx, iid in enumerate(item_ids)}

ratings_df['user_idx'] = ratings_df['user_id'].map(user_id_map)
ratings_df['item_idx'] = ratings_df['item_id'].map(item_id_map)

# Train/Test split (80/20)
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=42)

print(f"Train set: {len(train_df)} ratings")
print(f"Test set: {len(test_df)} ratings")

In [ ]:
# Créer la matrice user-item (train)
def create_user_item_matrix(df, n_users, n_items):
    """Créer une matrice dense user-item."""
    matrix = np.zeros((n_users, n_items))
    for _, row in df.iterrows():
        matrix[int(row['user_idx']), int(row['item_idx'])] = row['rating']
    return matrix

n_users = len(user_ids)
n_items = len(item_ids)

train_matrix = create_user_item_matrix(train_df, n_users, n_items)
print(f"Train matrix shape: {train_matrix.shape}")
print(f"Non-zero entries: {np.count_nonzero(train_matrix)}")

# Visualiser une petite partie de la matrice
plt.figure(figsize=(10, 8))
plt.imshow(train_matrix[:50, :50], cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Rating')
plt.title('Matrice User-Item (50 premiers users × 50 premiers items)\n(Noir = pas de rating)')
plt.xlabel('Item Index')
plt.ylabel('User Index')
plt.show()

## 3. User-Based Collaborative Filtering

Recommander basé sur les utilisateurs similaires.

In [ ]:
# Calculer la similarité cosine entre tous les utilisateurs
# Note: pour éviter les divisions par zéro, on ajoute un epsilon
user_similarity = cosine_similarity(train_matrix + 1e-9)

print(f"User similarity matrix shape: {user_similarity.shape}")
print(f"Similarité moyenne: {user_similarity.mean():.4f}")
print(f"Similarité min/max: {user_similarity.min():.4f} / {user_similarity.max():.4f}")

# Visualiser les similarités
plt.figure(figsize=(10, 8))
plt.imshow(user_similarity[:100, :100], cmap='coolwarm', vmin=-1, vmax=1, aspect='auto')
plt.colorbar(label='Similarité Cosine')
plt.title('Matrice de Similarité User-User (100 premiers utilisateurs)')
plt.xlabel('User Index')
plt.ylabel('User Index')
plt.show()

In [ ]:
def predict_user_based(train_matrix, user_similarity, user_idx, item_idx, k=20):
    """
    Prédire le rating pour (user_idx, item_idx) avec user-based CF.
    
    Args:
        train_matrix: matrice user-item (n_users, n_items)
        user_similarity: matrice de similarité (n_users, n_users)
        user_idx: index de l'utilisateur
        item_idx: index de l'item
        k: nombre de voisins à considérer
    """
    # Trouver les utilisateurs qui ont raté cet item
    users_who_rated = np.where(train_matrix[:, item_idx] > 0)[0]
    
    if len(users_who_rated) == 0:
        # Aucun utilisateur n'a raté cet item -> retourner la moyenne globale
        return train_matrix[train_matrix > 0].mean()
    
    # Similarités avec ces utilisateurs
    sims = user_similarity[user_idx, users_who_rated]
    
    # Prendre les k plus similaires
    top_k_indices = np.argsort(sims)[-k:][::-1]
    top_k_users = users_who_rated[top_k_indices]
    top_k_sims = sims[top_k_indices]
    
    # Ratings moyens
    user_mean = train_matrix[user_idx][train_matrix[user_idx] > 0].mean()
    neighbor_means = np.array([train_matrix[u][train_matrix[u] > 0].mean() for u in top_k_users])
    
    # Ratings des voisins pour cet item
    neighbor_ratings = train_matrix[top_k_users, item_idx]
    
    # Prédiction pondérée (mean-centered)
    if top_k_sims.sum() == 0:
        return user_mean
    
    prediction = user_mean + np.sum(top_k_sims * (neighbor_ratings - neighbor_means)) / np.sum(np.abs(top_k_sims))
    
    # Clip entre 1 et 5
    return np.clip(prediction, 1, 5)

# Test sur quelques prédictions
print("=== Test User-Based CF ===")
for i in range(5):
    row = test_df.iloc[i]
    user_idx = int(row['user_idx'])
    item_idx = int(row['item_idx'])
    true_rating = row['rating']
    pred_rating = predict_user_based(train_matrix, user_similarity, user_idx, item_idx, k=20)
    print(f"User {row['user_id']}, Item {row['item_id']}: True={true_rating:.1f}, Pred={pred_rating:.2f}")

In [ ]:
# Évaluation sur le test set (sur un échantillon pour accélérer)
print("\nÉvaluation User-Based CF (cela peut prendre quelques minutes)...")

# Prendre un échantillon du test set
test_sample = test_df.sample(n=min(1000, len(test_df)), random_state=42)

y_true = []
y_pred_user = []

for _, row in test_sample.iterrows():
    user_idx = int(row['user_idx'])
    item_idx = int(row['item_idx'])
    true_rating = row['rating']
    pred_rating = predict_user_based(train_matrix, user_similarity, user_idx, item_idx, k=20)
    
    y_true.append(true_rating)
    y_pred_user.append(pred_rating)

y_true = np.array(y_true)
y_pred_user = np.array(y_pred_user)

rmse_user = np.sqrt(mean_squared_error(y_true, y_pred_user))
mae_user = mean_absolute_error(y_true, y_pred_user)

print(f"\nUser-Based CF Results:")
print(f"RMSE: {rmse_user:.4f}")
print(f"MAE: {mae_user:.4f}")

## 4. Item-Based Collaborative Filtering

Recommander basé sur les items similaires.

In [ ]:
# Calculer la similarité cosine entre tous les items
item_similarity = cosine_similarity(train_matrix.T + 1e-9)

print(f"Item similarity matrix shape: {item_similarity.shape}")
print(f"Similarité moyenne: {item_similarity.mean():.4f}")

# Visualiser les similarités
plt.figure(figsize=(10, 8))
plt.imshow(item_similarity[:100, :100], cmap='coolwarm', vmin=-1, vmax=1, aspect='auto')
plt.colorbar(label='Similarité Cosine')
plt.title('Matrice de Similarité Item-Item (100 premiers items)')
plt.xlabel('Item Index')
plt.ylabel('Item Index')
plt.show()

In [ ]:
def predict_item_based(train_matrix, item_similarity, user_idx, item_idx, k=20):
    """
    Prédire le rating pour (user_idx, item_idx) avec item-based CF.
    
    Args:
        train_matrix: matrice user-item (n_users, n_items)
        item_similarity: matrice de similarité (n_items, n_items)
        user_idx: index de l'utilisateur
        item_idx: index de l'item
        k: nombre de voisins à considérer
    """
    # Trouver les items ratés par cet utilisateur
    items_rated = np.where(train_matrix[user_idx, :] > 0)[0]
    
    if len(items_rated) == 0:
        # L'utilisateur n'a raté aucun item -> retourner la moyenne globale
        return train_matrix[train_matrix > 0].mean()
    
    # Similarités avec ces items
    sims = item_similarity[item_idx, items_rated]
    
    # Prendre les k plus similaires
    top_k_indices = np.argsort(sims)[-k:][::-1]
    top_k_items = items_rated[top_k_indices]
    top_k_sims = sims[top_k_indices]
    
    # Ratings de l'utilisateur pour ces items
    user_ratings = train_matrix[user_idx, top_k_items]
    
    # Prédiction pondérée
    if top_k_sims.sum() == 0:
        return train_matrix[user_idx][train_matrix[user_idx] > 0].mean()
    
    prediction = np.sum(top_k_sims * user_ratings) / np.sum(np.abs(top_k_sims))
    
    # Clip entre 1 et 5
    return np.clip(prediction, 1, 5)

# Test sur quelques prédictions
print("=== Test Item-Based CF ===")
for i in range(5):
    row = test_df.iloc[i]
    user_idx = int(row['user_idx'])
    item_idx = int(row['item_idx'])
    true_rating = row['rating']
    pred_rating = predict_item_based(train_matrix, item_similarity, user_idx, item_idx, k=20)
    print(f"User {row['user_id']}, Item {row['item_id']}: True={true_rating:.1f}, Pred={pred_rating:.2f}")

In [ ]:
# Évaluation sur le test set
print("\nÉvaluation Item-Based CF...")

y_pred_item = []

for _, row in test_sample.iterrows():
    user_idx = int(row['user_idx'])
    item_idx = int(row['item_idx'])
    pred_rating = predict_item_based(train_matrix, item_similarity, user_idx, item_idx, k=20)
    y_pred_item.append(pred_rating)

y_pred_item = np.array(y_pred_item)

rmse_item = np.sqrt(mean_squared_error(y_true, y_pred_item))
mae_item = mean_absolute_error(y_true, y_pred_item)

print(f"\nItem-Based CF Results:")
print(f"RMSE: {rmse_item:.4f}")
print(f"MAE: {mae_item:.4f}")

## 5. Matrix Factorization avec SVD

Utilisons la bibliothèque **Surprise** pour une implémentation efficace de SVD.

In [ ]:
from surprise import SVD, NMF
from surprise.model_selection import cross_validate, GridSearchCV

# Préparer les données pour Surprise
trainset = data.build_full_trainset()

# Entraîner SVD
print("=== Entraînement SVD ===")
svd = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
svd.fit(trainset)

print("\nModèle SVD entraîné!")
print(f"Nombre de facteurs latents: {svd.n_factors}")
print(f"User embeddings shape: {svd.pu.shape}")
print(f"Item embeddings shape: {svd.qi.shape}")

In [ ]:
# Cross-validation
print("\n=== Cross-Validation (5-fold) ===")
cv_results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

print(f"\nRMSE moyen: {cv_results['test_rmse'].mean():.4f} ± {cv_results['test_rmse'].std():.4f}")
print(f"MAE moyen: {cv_results['test_mae'].mean():.4f} ± {cv_results['test_mae'].std():.4f}")

In [ ]:
# Évaluation sur test set
print("\nÉvaluation SVD sur test set...")

y_pred_svd = []

for _, row in test_sample.iterrows():
    user_id = row['user_id']
    item_id = row['item_id']
    pred = svd.predict(user_id, item_id)
    y_pred_svd.append(pred.est)

y_pred_svd = np.array(y_pred_svd)

rmse_svd = np.sqrt(mean_squared_error(y_true, y_pred_svd))
mae_svd = mean_absolute_error(y_true, y_pred_svd)

print(f"\nSVD Results:")
print(f"RMSE: {rmse_svd:.4f}")
print(f"MAE: {mae_svd:.4f}")

## 6. Comparaison des Approches

In [ ]:
# Tableau comparatif
results_df = pd.DataFrame({
    'Méthode': ['User-Based CF', 'Item-Based CF', 'SVD'],
    'RMSE': [rmse_user, rmse_item, rmse_svd],
    'MAE': [mae_user, mae_item, mae_svd]
})

print("\n=== Comparaison des Méthodes ===")
print(results_df.to_string(index=False))

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

methods = results_df['Méthode']
x_pos = np.arange(len(methods))

axes[0].bar(x_pos, results_df['RMSE'], color=['coral', 'steelblue', 'mediumseagreen'])
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(methods, rotation=15)
axes[0].set_ylabel('RMSE')
axes[0].set_title('RMSE par Méthode (plus bas = meilleur)')
axes[0].grid(axis='y', alpha=0.3)

axes[1].bar(x_pos, results_df['MAE'], color=['coral', 'steelblue', 'mediumseagreen'])
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(methods, rotation=15)
axes[1].set_ylabel('MAE')
axes[1].set_title('MAE par Méthode (plus bas = meilleur)')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots : prédictions vs vérité
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# User-Based
axes[0].scatter(y_true, y_pred_user, alpha=0.3, s=20)
axes[0].plot([1, 5], [1, 5], 'r--', lw=2)
axes[0].set_xlabel('Ratings Réels')
axes[0].set_ylabel('Ratings Prédits')
axes[0].set_title(f'User-Based CF\nRMSE={rmse_user:.3f}')
axes[0].grid(alpha=0.3)

# Item-Based
axes[1].scatter(y_true, y_pred_item, alpha=0.3, s=20, color='steelblue')
axes[1].plot([1, 5], [1, 5], 'r--', lw=2)
axes[1].set_xlabel('Ratings Réels')
axes[1].set_ylabel('Ratings Prédits')
axes[1].set_title(f'Item-Based CF\nRMSE={rmse_item:.3f}')
axes[1].grid(alpha=0.3)

# SVD
axes[2].scatter(y_true, y_pred_svd, alpha=0.3, s=20, color='mediumseagreen')
axes[2].plot([1, 5], [1, 5], 'r--', lw=2)
axes[2].set_xlabel('Ratings Réels')
axes[2].set_ylabel('Ratings Prédits')
axes[2].set_title(f'SVD\nRMSE={rmse_svd:.3f}')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Visualisation des Embeddings (SVD)

Visualisons les embeddings des films avec t-SNE.

In [ ]:
from sklearn.manifold import TSNE

# Récupérer les embeddings des items
item_embeddings = svd.qi  # (n_items, n_factors)

print(f"Item embeddings shape: {item_embeddings.shape}")

# Réduire à 2D avec t-SNE (sur un échantillon pour accélérer)
n_items_viz = min(500, item_embeddings.shape[0])
sample_indices = np.random.choice(item_embeddings.shape[0], n_items_viz, replace=False)
sample_embeddings = item_embeddings[sample_indices]

print(f"\nRéduction dimensionnalité avec t-SNE (cela peut prendre ~1 minute)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
embeddings_2d = tsne.fit_transform(sample_embeddings)

print("t-SNE terminé!")

In [ ]:
# Visualiser les embeddings
plt.figure(figsize=(12, 10))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.6, s=30, c='steelblue')
plt.title('Visualisation t-SNE des Embeddings de Films (SVD)\n500 films échantillonnés')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.grid(alpha=0.3)
plt.show()

print("\nLes films proches dans cet espace ont des embeddings similaires,")
print("ce qui signifie qu'ils sont ratés de façon similaire par les utilisateurs.")

## 8. Top-K Recommendations

Générons des recommandations top-10 pour un utilisateur.

In [ ]:
def recommend_top_k_svd(svd_model, user_id, train_df, k=10):
    """
    Générer les top-K recommandations pour un utilisateur.
    
    Args:
        svd_model: modèle SVD entraîné
        user_id: ID de l'utilisateur
        train_df: DataFrame des ratings d'entraînement
        k: nombre de recommandations
    """
    # Items déjà ratés par l'utilisateur
    rated_items = set(train_df[train_df['user_id'] == user_id]['item_id'])
    
    # Tous les items
    all_items = set(train_df['item_id'].unique())
    
    # Candidats = items non ratés
    candidates = all_items - rated_items
    
    # Prédire pour tous les candidats
    predictions = []
    for item_id in candidates:
        pred = svd_model.predict(user_id, item_id)
        predictions.append((item_id, pred.est))
    
    # Trier par score décroissant
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    return predictions[:k]

# Exemple pour un utilisateur
example_user_id = ratings_df['user_id'].iloc[0]

print(f"=== Recommandations Top-10 pour l'utilisateur {example_user_id} ===")
top_10 = recommend_top_k_svd(svd, example_user_id, train_df, k=10)

for rank, (item_id, score) in enumerate(top_10, 1):
    print(f"{rank}. Item {item_id}: score prédit = {score:.2f}")

# Montrer les films déjà ratés par cet utilisateur
user_ratings = train_df[train_df['user_id'] == example_user_id][['item_id', 'rating']].sort_values('rating', ascending=False)
print(f"\n=== Films déjà ratés par l'utilisateur {example_user_id} (top 5) ===")
print(user_ratings.head(5).to_string(index=False))

## Conclusion

Dans ce notebook, nous avons exploré différentes approches de **Collaborative Filtering** :

1. **User-Based CF** : basé sur les utilisateurs similaires
2. **Item-Based CF** : basé sur les items similaires
3. **Matrix Factorization (SVD)** : décomposition de la matrice en facteurs latents

**Résultats typiques** :
- **SVD** obtient généralement les meilleurs résultats (RMSE le plus bas)
- **Item-Based CF** est souvent meilleur que User-Based pour MovieLens
- Les approches neighborhood (user/item-based) sont plus explicables mais moins performantes

**Prochaines étapes** :
- Explorer les approches de **Deep Learning** (NCF, autoencoders)
- Évaluer avec des métriques de ranking (Precision@K, NDCG)
- Combiner avec du content-based filtering (systèmes hybrides)